# OLD

In [ ]:
import pandas as pd
import json

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("lc_quad", split= "train")

In [ ]:
train = [i if i['subgraph'] == 'string matching type + relation contains word' else None for i in dataset]

In [ ]:
train = list(filter(lambda x: x , train))

In [ ]:
train_df = pd.DataFrame.from_dict(train)

In [ ]:
train_df.head()

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660 entries, 0 to 659
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   NNQT_question         660 non-null    object
 1   uid                   660 non-null    int64 
 2   subgraph              660 non-null    object
 3   template_index        660 non-null    int64 
 4   question              660 non-null    object
 5   sparql_wikidata       660 non-null    object
 6   sparql_dbpedia18      660 non-null    object
 7   template              660 non-null    object
 8   paraphrased_question  660 non-null    object
dtypes: int64(2), object(7)
memory usage: 46.5+ KB


In [ ]:
train_df[['question', 'sparql_wikidata']].sample(n=10)

,question,sparql_wikidata
437,Name a non-science partially coincident with G...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
105,Which is the currency that has a location of f...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
643,what is former builiding in ancient greece cul...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
398,Name a multiple star constellation with Centau...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
216,Tell me the written work whose place of public...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
441,Tell me the security agency of chairperson Yur...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
379,What is the name of the crude drug that contai...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
282,Name a city located next to the Rhine river th...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
642,WHICH IS THE MINOR BASILICA THAT ARCHITECTURAL...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
332,What country whose archives are located in the...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...


Sparql API

In [ ]:
import requests

url = 'https://query.wikidata.org/sparql'
query = '''
SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj wdt:P31 wd:Q50053 . ?sbj wdt:P59 wd:Q10538 . ?sbj rdfs:label ?sbj_label . FILTER(STRSTARTS(lcase(?sbj_label), 's')) . FILTER (lang(?sbj_label) = 'en') } LIMIT 25
'''
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()
data['results']

{'bindings': [{'sbj': {'type': 'uri',
    'value': 'http://www.wikidata.org/entity/Q3409'},
   'sbj_label': {'xml:lang': 'en', 'type': 'literal', 'value': 'Sirius'}},
  {'sbj': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3962041'},
   'sbj_label': {'xml:lang': 'en', 'type': 'literal', 'value': 'Sirius B'}},
  {'sbj': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q13415844'},
   'sbj_label': {'xml:lang': 'en', 'type': 'literal', 'value': 'Sirius A'}}]}

In [ ]:
import re
import string
from collections import defaultdict

# import spacy
# from tqdm.notebook import tqdm
# nlp = spacy.load("en_core_web_sm")



In [ ]:
def join_spacy_tokens(tokens):
    ret = ""
    for t in tokens:
        if t[0] in string.punctuation:
            ret += t
        else:
            ret += (" " + t)
    return ret.strip()

In [ ]:
def negate_root_verb(nlp, q_body, rank):
    doc = nlp(q_body.strip())
    tokens = [token.text for token in doc]

    root_i = None
    root_token = None
    for i, token in enumerate(doc):
        if token.dep_ == "ROOT" and token.pos_ == "VERB":
            root_i = i
            root_token = token
            break
    """
    https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
    27. VB	Verb, base form
    28.	VBD	Verb, past tense
    29.	VBG	Verb, gerund or present participle
    30.	VBN	Verb, past participle
    31.	VBP	Verb, non-3rd person singular present
    32.	VBZ	Verb, 3rd person singular present
    """
    print(root_i)
    if root_i:
        if root_token.tag_ == "VBD":
            tokens = tokens[:root_i] + (["did", "not", root_token.lemma_] if rank % 2 else ["didn't", root_token.lemma_]) + tokens[root_i + 1:]
        elif root_token.tag_ == "VBP":
            tokens = tokens[:root_i] + (["do", "not", root_token.lemma_] if rank % 2 else ["don't", root_token.lemma_]) + tokens[root_i + 1:]
        elif root_token.tag_ == "VBZ":
            tokens = tokens[:root_i] + (["does", "not", root_token.lemma_] if rank % 2 else ["doesn't", root_token.lemma_]) + tokens[root_i + 1:]
        else:
            return None
    else:
        return None
    return join_spacy_tokens(tokens)

In [ ]:
from negate import Negator

# Use default model (en_core_web_md):

def negate_root_verb( q_body, rank):
  negator = Negator(use_transformers=True, use_gpu=True)
  negated_sentence = negator.negate_sentence(q_body)
  return negated_sentence if negated_sentence else None

In [ ]:
from copy import deepcopy

In [ ]:
cnt = 0
processed_questions = []
counters = defaultdict(int)
train_c = deepcopy(train)
for q in train_c:
    q_text = q["question"]
    orig = q_text
    q["original"] = orig
    if not q_text:
        continue
    q_text = q_text.replace("?", "")
    q_text = q_text.replace(".", "")
    q_text_split = q_text.split(" and ")
    q_body_old = q_text_split[-1].split(', ')[-1].strip()

    q_body = q_text_split[-1].split(', ')[-1].strip()
    q_words = q_body.split()

    q_body = " " + q_body

    if "Not" in q_words or "not" in q_words or "n't" in q_body\
        or "nothing" in q_words or "none" in q_words or "never" in q_words or "nobody" in q_words \
        or "nowhere" in q_words or "no" in q_words:
        q["negation_rule"] = "gold/not"

    elif "unable" in q_words or "unlike" in q_words or "unlikely" in q_words \
        or "incapable" in q_words or "unusual" in q_words or "impossible" in q_words \
        or "rarely" in q_words or "rare" in q_words or "barely" in q_words or "scarcely" in q_words:
        q["negation_rule"] = "gold/un-in-"

    elif "because" in q_words:
        q["negation_rule"] = "not/because"
        q_body = q_body.replace(" because", " not because")

    elif q_words.count("is") + q_words.count("are") + q_words.count("was") + q_words.count("were") == 1:
        q["negation_rule"] = "not/be"
        rank = counters[q["negation_rule"]]
        q_body = q_body.replace(" is", (" is not" if rank % 2 else " isn't"))
        q_body = q_body.replace(" are", (" are not" if rank % 2 else " aren't"))
        q_body = q_body.replace(" was", (" was not" if rank % 2 else " wasn't"))
        q_body = q_body.replace(" were", (" were not" if rank % 2 else " weren't"))

    elif (
            q_words.count("may") + q_words.count("might") \
            + q_words.count("can") + q_words.count("could") \
            + q_words.count("will") + q_words.count("would") == 1
        ):
        q["negation_rule"] = "not/modal"
        rank = counters[q["negation_rule"]]
        q_body = q_body.replace(" may", " may not")
        q_body = q_body.replace(" might", " might not")
        q_body = q_body.replace(" can", (" can not" if rank % 2 else " can't"))
        q_body = q_body.replace(" could", (" could not" if rank % 2 else " couldn't"))
        q_body = q_body.replace(" will", (" will not" if rank % 2 else " won't"))
        q_body = q_body.replace(" would", (" would not" if rank % 2 else " wouldn't"))

    elif negate_root_verb(q_body, counters["not/do"]):
        q["negation_rule"] = "not/do"
        rank = counters[q["negation_rule"]]
        q_body = negate_root_verb(q_body, rank)

    if "negation_rule" in q:
        q["question"] = q_text.replace(q_body_old.strip(), q_body.strip())

        processed_questions.append([q['original'], q['question']])
        counters[q["negation_rule"]] += 1


print(len(processed_questions))
print(dict(counters))
# print(processed_questions)

In [ ]:
f = open("b.txt", "w", encoding="utf-8")
for i, j in processed_questions:
  f.write("Question: " + i + "  ----  ")
  f.write("Negated: " + j )
  f.write("\n")
f.close()

In [ ]:
print(negate_root_verb( "contains the word model in it's name", 1))

0
None


In [ ]:
!pip install negate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.1 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.4
    Uninstalling spacy-3.5.4:
      Successfully uninstalled spacy-3.5.4
ERROR: pip's dependency resolver does not currently take into account all 

In [ ]:
from negate import Negator

# Use default model (en_core_web_md):
negator = Negator()


In [ ]:
negator = Negator(use_transformers=True, use_gpu=True)

ValueError: ignored

In [ ]:
sentence = "which has birth city"

negated_sentence = negator.negate_sentence(sentence)

print(negated_sentence)

Which doesn't have birth city


In [ ]:
def negate(sentence):
    negator = Negator(use_transformers=True, use_gpu=True)
    negated_sentence = negator.negate_sentence(sentence)
    return negated_sentence

In [ ]:
cnt = 0
processed_questions = []
counters = defaultdict(int)
train_c = deepcopy(train)
c = 0
for q in train_c:
    q_text = q["question"].lower()
    orig = q_text
    q["original"] = orig
    if not q_text:
        continue
    # q_text = q_text.replace("?", "")
    # q_text = q_text.replace(".", "")
    # q_text_split = q_text.split(" and ")
    # q_body_old = q_text_split[-1].split(', ')[-1].strip()

    # q_body = q_text_split[-1].split(', ')[-1].strip()
    # q_words = q_body.split()

    # q_body = " " + q_body

    # if "Not" in q_words or "not" in q_words or "n't" in q_body\
    #     or "nothing" in q_words or "none" in q_words or "never" in q_words or "nobody" in q_words \
    #     or "nowhere" in q_words or "no" in q_words:
    #     q["negation_rule"] = "gold/not"

    # elif "unable" in q_words or "unlike" in q_words or "unlikely" in q_words \
    #     or "incapable" in q_words or "unusual" in q_words or "impossible" in q_words \
    #     or "rarely" in q_words or "rare" in q_words or "barely" in q_words or "scarcely" in q_words:
    #     q["negation_rule"] = "gold/un-in-"

    # elif "because" in q_words:
    #     q["negation_rule"] = "not/because"
    #     q_body = q_body.replace(" because", " not because")

    # elif q_words.count("is") + q_words.count("are") + q_words.count("was") + q_words.count("were") == 1:
    #     q["negation_rule"] = "not/be"
    #     rank = counters[q["negation_rule"]]
    #     q_body = q_body.replace(" is", (" is not" if rank % 2 else " isn't"))
    #     q_body = q_body.replace(" are", (" are not" if rank % 2 else " aren't"))
    #     q_body = q_body.replace(" was", (" was not" if rank % 2 else " wasn't"))
    #     q_body = q_body.replace(" were", (" were not" if rank % 2 else " weren't"))

    # elif (
    #         q_words.count("may") + q_words.count("might") \
    #         + q_words.count("can") + q_words.count("could") \
    #         + q_words.count("will") + q_words.count("would") == 1
    #     ):
    #     q["negation_rule"] = "not/modal"
    #     rank = counters[q["negation_rule"]]
    #     q_body = q_body.replace(" may", " may not")
    #     q_body = q_body.replace(" might", " might not")
    #     q_body = q_body.replace(" can", (" can not" if rank % 2 else " can't"))
    #     q_body = q_body.replace(" could", (" could not" if rank % 2 else " couldn't"))
    #     q_body = q_body.replace(" will", (" will not" if rank % 2 else " won't"))
    #     q_body = q_body.replace(" would", (" would not" if rank % 2 else " wouldn't"))

    # elif negate_root_verb(q_body, counters["not/do"]):
    q["negation_rule"] = "not/do"
    rank = counters[q["negation_rule"]]
    q_text = q_text.replace("start", "does not start")
    if q_text == orig:
      q_text = q_text.replace("contain", "does not contain")
    if q_text == orig:
      q_text = q_text.replace("begin", "does not begin")
    if q_text == orig:
      q_text = q_text.replace("consist", "does not consist")
    if q_text != orig:
      c += 1

    if "negation_rule" in q:
        q["question"] = q_text
        processed_questions.append([q['original'], q['question']])
        counters[q["negation_rule"]] += 1
    else:
        q["question"] = ""
        processed_questions.append([q['original'], q['question']])

print(len(processed_questions))
print(dict(counters))
print(c)

660
{'not/do': 660}
613


# NEW

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("lc_quad", split= "train")

Generating train split:   0%|          | 0/19293 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4781 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd

In [ ]:
train = [i if i['subgraph'] == 'string matching type + relation contains word' else None for i in dataset]

In [ ]:
train = list(filter(lambda x: x , train))

In [ ]:
train_df = pd.DataFrame.from_dict(train)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660 entries, 0 to 659
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   NNQT_question         660 non-null    object
 1   uid                   660 non-null    int64 
 2   subgraph              660 non-null    object
 3   template_index        660 non-null    int64 
 4   question              660 non-null    object
 5   sparql_wikidata       660 non-null    object
 6   sparql_dbpedia18      660 non-null    object
 7   template              660 non-null    object
 8   paraphrased_question  660 non-null    object
dtypes: int64(2), object(7)
memory usage: 46.5+ KB


In [ ]:
train_df

,NNQT_question,uid,subgraph,template_index,question,sparql_wikidata,sparql_dbpedia18,template,paraphrased_question
0,Give me {female beauty pageant} that {operatin...,25156,string matching type + relation contains word,1095,Tell me the female beauty pageant that operate...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...,SELECT DISTINCT ?sbj ?sbj_label { ?statement1 ...,<?S P O ; ?S instanceOf Type ; contains word >,Tell me the female splendor festival that oper...
1,Give me {cause of death} that {medical examina...,25596,string matching type + relation contains word,453,What is a cause of death that begins with the ...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...,SELECT DISTINCT ?sbj ?sbj_label { ?statement1 ...,<?S P O ; ?S instanceOf Type ; starts with ch...,What is a purpose of dying that begins with th...
2,Give me {film character} that {from fictional ...,25365,string matching type + relation contains word,3185,Give me a film character from a fictional univ...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...,SELECT DISTINCT ?sbj ?sbj_label { ?statement1 ...,<?S P O ; ?S instanceOf Type ; starts with ch...,Give me a movie personality from a fictional u...
3,Give me {empire} that {coat of arms} {royal co...,24949,string matching type + relation contains word,3209,"Name an empire that contains the word ""british...",SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...,SELECT DISTINCT ?sbj ?sbj_label { ?statement1 ...,<?S P O ; ?S instanceOf Type ; contains word >,"Title an realm that contains the word ""british..."
4,Give me {Greek deity} that {sibling} {Zeus} an...,25085,string matching type + relation contains word,3714,Tell me the Greek deity of sibling of Zeus and...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...,SELECT DISTINCT ?sbj ?sbj_label { ?statement1 ...,<?S P O ; ?S instanceOf Type ; contains word >,Let me know the Greek deity of kin of Zeus whi...
...,...,...,...,...,...,...,...,...,...
655,Give me {type of number} that {disjoint union ...,25471,string matching type + relation contains word,3242,What what is type of number disjoint union of}...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...,SELECT DISTINCT ?sbj ?sbj_label { ?statement1 ...,<?S P O ; ?S instanceOf Type ; starts with ch...,What what is kind of number disjoint union of}...
656,Give me {media franchise} that {fictional univ...,25006,string matching type + relation contains word,2642,What is the name of the fictional universe des...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...,SELECT DISTINCT ?sbj ?sbj_label { ?statement1 ...,<?S P O ; ?S instanceOf Type ; contains word >,What is the identify of the fictional universe...
657,Give me {literary movement} that {partially co...,25623,string matching type + relation contains word,1277,Tell me the literary movement which is partial...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...,SELECT DISTINCT ?sbj ?sbj_label { ?statement1 ...,<?S P O ; ?S instanceOf Type ; starts with ch...,Tell me the literary motion which is partly co...
658,Give me {computer model} that {collection} {Mu...,25449,string matching type + relation contains word,2178,what is computer model collection by museum of...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...,SELECT DISTINCT ?sbj ?sbj_label { ?statement1 ...,<?S P O ; ?S instanceOf Type ; starts with ch...,what is computer demonstrate collection by gal...


In [ ]:
train_df[['question', 'sparql_wikidata']].sample(n=10)

,question,sparql_wikidata
325,Tell me the major party which holds the office...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
139,Name an art genre that starts with letter B,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
209,Name a space station that starts with letter S,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
399,Name a colonial Empire that starts with letter B,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
572,What what is divide and conquer algorithm disc...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
579,Which is the patent medicine that was invented...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
416,Name a song composed by Michael Jackson that c...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
108,What what is enterprise category of associated...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
509,"WHat is the association whose motto is Citius,...",SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
417,Name a policy known as a manifestation of raci...,SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...


# T

In [ ]:
import spacy

In [ ]:

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
sentences=["what is a cause of death that begins with the letter p and can be found on a ct scan?","what is municipality of Spain contains settlement Bobadilla and also which that starts with g","Name a region in China whose highest point is K2 that starts with letter X", "Who is the human biblical figure that is a significant person of Samuel and contains the word saul in it's name?"]

In [ ]:
def get_subject_phrase(doc):
    for token in doc:
        if ("subj" in token.dep_):
            subtree = list(token.subtree)
            start = subtree[0].i
            end = subtree[-1].i + 1
    return doc[start:end]

In [ ]:
def get_object_phrase(doc):
    for token in doc:
        if ("dobj" in token.dep_):
            subtree = list(token.subtree)
            start = subtree[0].i
            end = subtree[-1].i + 1
            return doc[start:end]

In [ ]:
for sentence in sentences:
    doc = nlp(sentence)
    for token in doc:
        ancestors = [t.text for t in token.ancestors]
        children = [t.text for t in token.children]
        print(token.text, "\t", token.i, "\t",
            token.pos_, "\t", token.dep_, "\t",
            ancestors, "\t", children)
    print()

what 	 0 	 PRON 	 attr 	 ['is'] 	 []
is 	 1 	 AUX 	 ROOT 	 [] 	 ['what', 'cause', '?']
a 	 2 	 DET 	 det 	 ['cause', 'is'] 	 []
cause 	 3 	 NOUN 	 nsubj 	 ['is'] 	 ['a', 'of', 'begins']
of 	 4 	 ADP 	 prep 	 ['cause', 'is'] 	 ['death']
death 	 5 	 NOUN 	 pobj 	 ['of', 'cause', 'is'] 	 []
that 	 6 	 PRON 	 nsubj 	 ['begins', 'cause', 'is'] 	 []
begins 	 7 	 VERB 	 relcl 	 ['cause', 'is'] 	 ['that', 'with', 'and', 'found']
with 	 8 	 ADP 	 prep 	 ['begins', 'cause', 'is'] 	 ['p']
the 	 9 	 DET 	 det 	 ['p', 'with', 'begins', 'cause', 'is'] 	 []
letter 	 10 	 NOUN 	 compound 	 ['p', 'with', 'begins', 'cause', 'is'] 	 []
p 	 11 	 NOUN 	 pobj 	 ['with', 'begins', 'cause', 'is'] 	 ['the', 'letter']
and 	 12 	 CCONJ 	 cc 	 ['begins', 'cause', 'is'] 	 []
can 	 13 	 AUX 	 aux 	 ['found', 'begins', 'cause', 'is'] 	 []
be 	 14 	 AUX 	 auxpass 	 ['found', 'begins', 'cause', 'is'] 	 []
found 	 15 	 VERB 	 conj 	 ['begins', 'cause', 'is'] 	 ['can', 'be', 'on']
on 	 16 	 ADP 	 prep 	 ['found', 'begin

In [ ]:
def find_root_of_sentence(doc):
    root_token = None
    for token in doc:
        if (token.dep_ == "ROOT"):
            root_token = token
    return root_token

In [ ]:
def find_other_verbs(doc, root_token):
    other_verbs = []
    for token in doc:
        ancestors = list(token.ancestors)
        if (token.pos_ == "VERB" and len(ancestors) == 1\
            and ancestors[0] == root_token):
            other_verbs.append(token)
    return other_verbs

In [ ]:
def get_clause_token_span_for_verb(verb, doc, all_verbs):
    first_token_index = len(doc)
    last_token_index = 0
    this_verb_children = list(verb.children)
    for child in this_verb_children:
        if (child not in all_verbs):
            if (child.i < first_token_index):
                first_token_index = child.i
            if (child.i > last_token_index):
                last_token_index = child.i
    return(first_token_index, last_token_index)

In [ ]:
for sentence in sentences:
    doc = nlp(sentence)
    root_token = find_root_of_sentence(doc)
    other_verbs = find_other_verbs(doc, root_token)
    token_spans = []
    all_verbs = [root_token] + other_verbs
    for other_verb in all_verbs:
        (first_token_index, last_token_index) = \
         get_clause_token_span_for_verb(other_verb,
                                        doc, all_verbs)
        token_spans.append((first_token_index,
                            last_token_index))
    sentence_clauses = []
    for token_span in token_spans:
        start = token_span[0]
        end = token_span[1]
        if (start < end):
            clause = doc[start:end]
            sentence_clauses.append(clause)
    sentence_clauses = sorted(sentence_clauses,
                              key=lambda tup: tup[0])
    clauses_text = [clause.text for clause in sentence_clauses]
    print(clauses_text)

['is municipality of Spain contains settlement', 'which that starts']
['region', 'point is', 'that starts']
["Who is the human biblical figure that is a significant person of Samuel and contains the word saul in it's name"]


In [ ]:
stop_words = set(stopwords.words('english'))
for sentence in sentences:
    doc = nlp(sentence)
    for i in doc:
      if i.is_stop:
        continue
      print(i, i.tag_)
    # print(doc.ents)
    # subject_phrase = get_subject_phrase(doc)
    # object_phrase = get_object_phrase(doc)
    # print(subject_phrase)
    # print(object_phrase)

municipality NN
Spain NNP
contains VBZ
settlement NN
Bobadilla NNP
starts VBZ
g NN
region NN
China NNP
highest JJS
point NN
K2 NNP
starts VBZ
letter NN
X NN
human JJ
biblical JJ
figure NN
significant JJ
person NN
Samuel NNP
contains VBZ
word NN
saul NNP
? .


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

sentence = "This is a sample sentence."
encoded_dict = tokenizer.encode_plus(
                        sentence,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )

input_ids = encoded_dict['input_ids']
attention_mask = encoded_dict['attention_mask']
new = "It is a sentence and I am walking down the street"

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
import spacy

nlp = spacy.load("en_core_web_md")

def extract_similar_phrase(sentence, given_phrase):
    doc = nlp(sentence)

    similarity_scores = []
    for token in doc:
        if not token.is_stop:
          similarity = token.similarity(nlp(given_phrase))
          similarity_scores.append((token.text, similarity))

    # Sort the tokens by similarity score in descending order
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    print(similarity_scores)
    # Extract the most similar token
    extracted_phrase = similarity_scores[0][0] if similarity_scores else None
    return extracted_phrase

sentence = "What association football league that starts with the letter s is organized by the Italian Football Federation?"
given_phrase = "starts with letter X"

extracted_phrase = extract_similar_phrase(sentence, given_phrase)
print(extracted_phrase)


[('letter', 0.3851098465679654), ('starts', 0.335380224413214), ('association', 0.21779104083345924), ('Federation', 0.16328760738833545), ('organized', 0.13623614859581756), ('s', 0.10534495599057224), ('Italian', 0.08669379726629431), ('league', 0.07840338094018742), ('Football', 0.020549601159670146), ('football', -0.004523158947162801), ('?', -0.060817285961479284)]
letter


In [ ]:
!pip install pysvgview

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for yattag: filename=yattag-1.15.1-py3-none-any.whl size=15630 sha256=de0a61d479b8c310ab99d8723e60a786a66634f7ed1ff583836576facbeed906
  Stored in directory: /root/.cache/pip/wheels/c0/f7/67/89165fb6c0e73ad142c1a60daed9e773af08961396d06fa391
Successfully built yattag


In [ ]:
import spacy
from spacy import displacy
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
nlp = spacy.load("en_core_web_sm")
sentence = tokenizer.tokenize("Who is the human biblical figure that is a significant person of Samuel and contains the word saul in it's name?")
sentence = ' '.join(sentence)
doc = nlp(sentence)
displacy.render(doc, style="dep", jupyter=True)

In [ ]:
negator = Negator(fail_on_unsupported=True)


In [ ]:
import spacy
from spacy import displacy
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

# sentences=["what is a cause of death that begins with the letter p and can be found on a ct scan?","what is municipality of Spain contains settlement Bobadilla and also which that starts with g","Name a region in China whose highest point is K2 that starts with letter X", "Who is the human biblical figure that is a significant person of Samuel and contains the word saul in it's name?"]
nlp = spacy.load("en_core_web_sm")
for s in range(len(train_df["question"])):
    sentence = train_df["question"][s]
    sentence = tokenizer.tokenize(sentence)
    sentence = ' '.join(sentence)
    doc = nlp(sentence)
    r = None
    for token in doc:
      if token.pos_ == "VERB":
        r = token
    if r:
      try:
        token = r
        ancestors = [t for t in token.ancestors]
        children = [t for t in token.children]
        # print(token.text, "\t", token.i, "\t",
            # token.pos_, "\t", token.dep_, "\t",
            # ancestors, "\t", children)
        if children:
          l = min(children[0].i, token.i)
          r = max(children[-1].i, token.i) + 1
        else:
          l = token.i
          r = token.i + 1
        pt = doc[l : r]
        pt_str = " ".join([i.text for i in pt])
        Neg = negator.negate_sentence(pt_str)
        print("Org : ",sentence)
        print("Prt : ",pt_str)
        sentence = sentence.replace(pt_str, Neg)
        print("Neg : ",sentence)
        print()
      except RuntimeError:
        pass

Org :  Tell me the female beauty pageant that operates in all countries and contains the word model in it s name
Prt :  contains the word model in
Neg :  Tell me the female beauty pageant that operates in all countries and Doesn't contain the word model in it s name

Org :  What is a cause of death that begins with the letter p and can be found on a CT scan
Prt :  can be found on
Neg :  What is a cause of death that begins with the letter p and Can't be found on a CT scan

Org :  Give me a film character from a fictional universe such as Marvel comics that starts with a W
Prt :  that starts with
Neg :  Give me a film character from a fictional universe such as Marvel comics That doesn't start with a W

Org :  Name an empire that contains the word british in its name
Prt :  that contains the word british
Neg :  Name an empire That doesn't contain the word british in its name

Org :  Tell me the Greek deity of sibling of Zeus and that contains the word poseidon in it s name
Prt :  that c

Org :  tell me literary character first appearance Action Comics 1 starts with s
Prt :  appearance Action Comics 1 starts with
Neg :  tell me literary character first Appearance Action Comics 1 doesn't start with s

Org :  Tell me the software company whose stock exchange is Tokyo Stock Exchange and which that starts with the letter n
Prt :  which that starts with
Neg :  Tell me the software company whose stock exchange is Tokyo Stock Exchange and Which that doesn't start with the letter n

Org :  Which unit of time starts with the letter y and measures physical quantity or duration
Prt :  measures physical quantity
Neg :  Which unit of time starts with the letter y and Doesn't measure physical quantity or duration

Org :  Name this nationalist party in India that starts with letter P
Prt :  that starts with
Neg :  Name this nationalist party in India That doesn't start with letter P

Org :  Tell me the mausoleum commissioned by Shah Jahan and whose name contains the word mahal
Prt :  

In [ ]:
queries = train_df['sparql_wikidata']

In [ ]:
import re
train_df['sparql_fp'] = queries.apply(lambda x: re.sub(r'. FILTER.*\.', '.', x))
train_df['sparql_fp']

0      SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
1      SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
2      SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
3      SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
4      SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
                             ...                        
655    SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
656    SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
657    SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
658    SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
659    SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj w...
Name: sparql_fp, Length: 660, dtype: object

In [ ]:
train_df['sparql_neg'] = queries.apply(lambda x: re.sub(r'STRSTARTS', '!STRSTARTS', x))

In [ ]:
train_df['sparql_neg'] = train_df['sparql_neg'].apply(lambda x: re.sub(r'CONTAINS', '!CONTAINS', x))

In [ ]:
import requests
import time
sparql_out = []
c = 0
url = 'https://query.wikidata.org/sparql'
for query in train_df['sparql_fp']:
    print(c)
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
    sparql_out.append(data)
    c += 1
    time.sleep(2)

In [ ]:
import requests
sparql_neg_out = []
c = 0
url = 'https://query.wikidata.org/sparql'
for query in train_df['sparql_neg']:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    print(c)
    data = r.json()
    sparql_neg_out.append(data)
    c += 1
    time.sleep(2)

In [ ]:
import requests
sparql_org_out = []
c = 0
url = 'https://query.wikidata.org/sparql'
for query in train_df['sparql_wikidata']:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    print(c)
    data = r.json()
    sparql_org_out.append(data)
    c += 1
    time.sleep(2)

In [ ]:
t = 0
for i, j in enumerate(train_df['question']):
    v = sparql_out[i]['results']['bindings']
    if len(v) > 1 and len(v) < 11:
        sentence = train_df["question"][i]
        sentence = tokenizer.tokenize(sentence)
        sentence = ' '.join(sentence)
        doc = nlp(sentence)
        r = None
        for token in doc:
          if token.pos_ == "VERB":
            r = token
        if r:
          try:
            token = r
            ancestors = [t for t in token.ancestors]
            children = [t for t in token.children]
            # print(token.text, "\t", token.i, "\t",
                # token.pos_, "\t", token.dep_, "\t",
                # ancestors, "\t", children)
            if children:
              l = min(children[0].i, token.i)
              r = max(children[-1].i, token.i) + 1
            else:
              l = token.i
              r = token.i + 1
            pt = doc[l : r]
            pt_str = " ".join([i.text for i in pt])
            Neg = negator.negate_sentence(pt_str)
            print("Org : ",sentence)
            print("Prt : ",pt_str)
            sentence = sentence.replace(pt_str, Neg)
            print("Neg : ",sentence)
          except RuntimeError:
            continue
        t += 1
        print("Output 1st part: ", end= " ")
        for x in v:
            print( x['sbj_label']['value'], end=", ")
        print()
        print("Output original query : ", end = " ")
        data = sparql_org_out[i]['results']['bindings']
        for x in data:
            print( x['sbj_label']['value'], end=", ")
        print()
        print("Output Negated original query : ", end = " ")
        data = sparql_neg_out[i]['results']['bindings']
        for x in data:
            print( x['sbj_label']['value'], end=", ")
        print()
        print()

Org :  What is a cause of death that begins with the letter p and can be found on a CT scan
Prt :  can be found on
Neg :  What is a cause of death that begins with the letter p and Can't be found on a CT scan
Output 1st part:  pneumonia, acute respiratory distress syndrome, 
Output original query :  pneumonia, 
Output Negated original query :  acute respiratory distress syndrome, 

Org :  Tell me the Greek deity of sibling of Zeus and that contains the word poseidon in it s name
Prt :  that contains the word poseidon in it s name
Neg :  Tell me the Greek deity of sibling of Zeus and That doesn't contain the word poseidon in it s name
Output 1st part:  Hestia, Hera, Poseidon, Demeter, Hades, 
Output original query :  Poseidon, 
Output Negated original query :  Hera, Demeter, Hades, Hestia, 

Org :  Name an alcoholic beverage that contains the word rum in its name
Prt :  that contains the word rum
Neg :  Name an alcoholic beverage That doesn't contain the word rum in its name
Output 1st 

In [ ]:
print(t)

96


In [ ]:
import pickle


with open('sparql_out.pickle', 'wb') as handle:
    pickle.dump(sparql_out, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle


with open('sparql_neg_out.pickle', 'wb') as handle:
    pickle.dump(sparql_neg_out, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle


with open('final_org.pickle', 'wb') as handle:
    pickle.dump(final_org, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: ignored

In [ ]:
train = [i if i['subgraph'] == 'right-subgraph' else None for i in dataset]

In [ ]:
train = list(filter(lambda x: x , train))

In [ ]:
train_df = pd.DataFrame.from_dict(train)

In [ ]:
train_df[30:40]

,NNQT_question,uid,subgraph,template_index,question,sparql_wikidata,sparql_dbpedia18,template,paraphrased_question
30,What is {is in the settlement of} of {British ...,13341,right-subgraph,1246,Name the BAFTA settlement with the postcode DA.,SELECT ?answer WHERE { wd:Q159661 wdt:P131 ?an...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF ?F . ?F RFG G,Title the BAFTA settlement with the postcode DA.
31,What is {religious affiliation} of {child of} ...,16491,right-subgraph,252,What is the religious affiliation of Vladimir ...,SELECT ?answer WHERE { wd:Q14974 wdt:P22 ?X . ...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF xF . xF RFG ?G,What is the spiritual affiliation of Vladimir ...
32,"What is {on harbour} of {Kryvyi Rih}, that has...",13156,right-subgraph,2761,"""Where is {on harbour} of {Kryvyi Rih}, which ...",SELECT ?answer WHERE { wd:Q157137 wdt:P206 ?an...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF ?F . ?F RFG G,"""Where is {on harbor} of {Kryvyi Rih}, which h..."
33,What is {service branch} of {Julius and Ethel ...,12102,right-subgraph,107,What is the service branch by Julius and Ethe...,SELECT ?answer WHERE { wd:Q769097 wdt:P241 ?an...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF ?F . ?F RFG G,What is the benefit department by Julius and E...
34,What is {manifestation of} of {Christmas tree}...,13651,right-subgraph,73,What appearance of a Christmas tree happens ev...,SELECT ?answer WHERE { wd:Q47128 wdt:P1557 ?an...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF ?F . ?F RFG G,What appearance of a Christmas tree happens ea...
35,"What is {developer} of {proper motion}, that h...",11891,right-subgraph,95,Who proved the theory of Proper Motion? Their ...,SELECT ?answer WHERE { wd:Q193223 wdt:P61 ?ans...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF ?F . ?F RFG G,Who proved Proper Motion's theory? Halley is t...
36,"What is {second language} of {Peter Faber}, th...",11833,right-subgraph,390,What is the second language of Peter Faber whi...,SELECT ?answer WHERE { wd:Q454814 wdt:P1412 ?a...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF ?F . ?F RFG G,What is the 2nd language of Peter Faber which ...
37,"What is {honorary title} of {Albert Günther}, ...",13322,right-subgraph,2173,What is the honorary title given to Albert Gun...,SELECT ?answer WHERE { wd:Q57514 wdt:P166 ?ans...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF ?F . ?F RFG G,What is the privileged title given to Albert G...
38,What is {located on astronomical body} of {Nea...,11906,right-subgraph,19,"What has the eccentricity of 0.0567, that is l...",SELECT ?answer WHERE { wd:Q782867 wdt:P376 ?an...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF ?F . ?F RFG G,"What has the eccentricity of 0.0567, that is p..."
39,What is {diplomatic relation} of {basin countr...,15632,right-subgraph,4911,What is the diplomatic relations like in basin...,SELECT ?answer WHERE { wd:Q202905 wdt:P205 ?X ...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF xF . xF RFG ?G,The basin countries of Lake Okeechobee has wha...


In [ ]:
import spacy
from spacy import displacy
from nltk.tokenize import RegexpTokenizer
import requests
import json
import time
url = 'https://query.wikidata.org/sparql'
tokenizer = RegexpTokenizer(r'\w+')

nlp = spacy.load("en_core_web_sm")

In [ ]:
final_fp = []
final_neg = []
final_org = []
final_ques_neg = []
for s in range(len(train_df["question"])):
    final_fp.append(None)
    final_neg.append(None)
    final_org.append(None)
    final_ques_neg.append(None)
    if train_df['sparql_wikidata'][s].find("?X") != -1:
      continue
    sentence = train_df["question"][s]
    sentence = tokenizer.tokenize(sentence)
    sentence = ' '.join(sentence)
    doc = nlp(sentence)
    r = None
    for token in doc:
      if token.pos_ == "VERB":
        r = token
    if r:
      try:
        token = r
        ancestors = [t for t in token.ancestors]
        children = [t for t in token.children]
        # print(token.text, "\t", token.i, "\t",
            # token.pos_, "\t", token.dep_, "\t",
            # ancestors, "\t", children)
        if children:
          l = min(children[0].i, token.i)
          r = max(children[-1].i, token.i) + 1
        else:
          l = token.i
          r = token.i + 1
        pt = doc[l : r]
        pt_str = " ".join([i.text for i in pt])
        Neg = negator.negate_sentence(pt_str)
        out_fp = requests.get(url, params = {'format': 'json', 'query': train_df['sparql_fp'][s]}).json()['results']['bindings']
        time.sleep(2)
        if len(out_fp) > 1 and len(out_fp) < 11:
          pass
        else:
          raise RuntimeError()
        out_neg = requests.get(url, params = {'format': 'json', 'query': train_df['sparql_neg'][s]}).json()['results']['bindings']
        time.sleep(2)
        if len(out_neg) > 0:
          pass
        else:
          raise RuntimeError()
        out_org = requests.get(url, params = {'format': 'json', 'query': train_df['sparql_wikidata'][s]}).json()['results']['bindings']
        time.sleep(2)
        if len(out_org) > 0:
          pass
        else:
          raise RuntimeError()
        out_names_fp = []
        out_names_neg = []
        out_names_org = []
        print(s,".")
        print("Org : ",sentence)
        print("Prt : ",pt_str)
        sentence = sentence.replace(pt_str, Neg)
        print("Neg : ",sentence)
        print("Output 1st part: ", end= " ")
        for x in out_fp:
            html = requests.get(x['answer']['value'], params={'format': "html"})
            time.sleep(2)
            soup = BeautifulSoup(html.text, 'html.parser')
            elements = soup.select("#firstHeading > span > span.wikibase-title-label")
            if not elements:
              continue
            out_names_fp.append(elements[0].text)
            print( elements[0].text, end=", ")
        print()
        print("Output original query : ", end = " ")
        for x in out_org:
            html = requests.get(x['answer']['value'], params={'format': "html"})
            time.sleep(2)
            soup = BeautifulSoup(html.text, 'html.parser')
            elements = soup.select("#firstHeading > span > span.wikibase-title-label")
            if not elements:
              continue
            out_names_neg.append(elements[0].text)
            print( elements[0].text, end=", ")
        print()
        print("Output Negated original query : ", end = " ")
        for x in out_neg:
            html = requests.get(x['answer']['value'], params={'format': "html"})
            time.sleep(2)
            soup = BeautifulSoup(html.text, 'html.parser')
            elements = soup.select("#firstHeading > span > span.wikibase-title-label")
            if not elements:
              continue
            out_names_org.append(elements[0].text)
            print(elements[0].text, end=", ")
        print()
        print()
        final_fp[s] = out_names_fp.copy()
        final_neg[s] = out_names_neg.copy()
        final_org[s] = out_names_org.copy()
        final_ques_neg[s] = sentence
      except RuntimeError:
        pass

1 .
Org :  Which is landscape of of Virgin of the rocks which has birth city is Tzippori
Prt :  which has birth city
Neg :  Which is landscape of of Virgin of the rocks Which doesn't have birth city is Tzippori
Output 1st part:  Virgin Mary, woman, boy, mother, angel, Christ Child, 
Output original query :  Virgin Mary, 
Output Negated original query :  angel, Christ Child, woman, boy, mother, 

4 .
Org :  What kind of career does Grigori Kozintsev have in the screenwriting field
Prt :  kind of career does Grigori Kozintsev have in
Neg :  What Kind of career doesn't Grigori Kozintsev have in the screenwriting field
Output 1st part:  screenwriter, film director, film producer, theatrical director, director, 
Output original query :  screenwriter, 
Output Negated original query :  film director, film producer, theatrical director, director, 

7 .
Org :  Which borders of borders of Riverside have a start date of 1984 0 0
Prt :  borders of borders of Riverside have a start date of 1984 0 0

In [ ]:
queries = train_df['sparql_wikidata']

In [ ]:
import re
train_df['sparql_fp'] = queries.apply(lambda x: re.sub(r'\. ?.*$', '. }', x))
train_df['sparql_fp'][10]

'SELECT ?answer WHERE { wd:Q1077445 wdt:P840 ?answer . }'

In [ ]:
train_df['sparql_neg'] = queries.apply(lambda x: re.sub(r'\. ?', '. MINUS { ', x))
train_df['sparql_neg'] = train_df['sparql_neg'].apply(lambda x: re.sub(r'}', ' }}', x))

In [ ]:
train_df['sparql_neg'][10]

'SELECT ?answer WHERE { wd:Q1077445 wdt:P840 ?answer . MINUS { ?answer wdt:P47 wd:Q49143 }}'

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
html = requests.get("http://www.wikidata.org/entity/Q345", params={'format': "html"})
soup = BeautifulSoup(html.text, 'html.parser')
elements = soup.select("#firstHeading > span > span.wikibase-title-label")
elements[0].text


'Virgin Mary'

In [ ]:
print(len(final_fp))

NameError: ignored